<center>
<h1>Chapter 14</h1>
</center>

<hr>

Web page (frontend) we will show user. Will work later on backend server in production notebook.

###This chapter is a bit schizofrenic

I am asking you to load all the Titanic pieces we have accumulated over past chapters. However, I am then asking you to change the frontend to be Pima-centric.

What gives? How can I have Titanic data mixed with Pima frontend? It's all under the umbrella of Pedagogy :)

We will not attempt to connect the frontend with the backend in this notebook. I'm asking you to build the back-end (Titanic data) just to make sure you can do it smoothly. Your challenges are focused on understanding the frontend. And I somewhat aribtarily chose the Pima data is where to practice.

Bottomline: for your final project you will have a unified backend and frontend. In this notebook, it is just practicing with the separate pieces.

##Set-up

First bring in your library.

In [1]:
github_name = 'marvnc'
repo_name = 'cs523'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file

rm: cannot remove 'library.py': No such file or directory
--2025-06-04 05:40:52--  https://raw.githubusercontent.com/marvnc/cs523/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48297 (47K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  47.17K  --.-KB/s    in 0.01s   

2025-06-04 05:40:53 (3.86 MB/s) - ‘library.py’ saved [48297/48297]



##You can load my files just for this noteook

In your final project, you will need to replace all my paths and urls with your own. You will be building these up in the final project notebooks.

##Load the files



In [2]:
from joblib import load
import tensorflow as tf

##Set-up pipeline documentation

Assumes markdown description on GitHub. Will load it then convert it to html. Can later insert into `fpage`.

In [3]:
#url to Pipeline documentation in markup format.
pipe_doc_url = 'https://raw.githubusercontent.com/fickas/course_datasets/refs/heads/main/pipeline-documentation.md'
!rm 'pipeline-documentation.md'
!wget $pipe_doc_url
with open('pipeline-documentation.md', 'r', encoding='utf-8') as file:
    pipe_md_content = file.read()

rm: cannot remove 'pipeline-documentation.md': No such file or directory
--2025-06-04 05:41:11--  https://raw.githubusercontent.com/fickas/course_datasets/refs/heads/main/pipeline-documentation.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4169 (4.1K) [text/plain]
Saving to: ‘pipeline-documentation.md’

pipeline-documentat 100%[===================>]   4.07K  --.-KB/s    in 0s      

2025-06-04 05:41:11 (41.7 MB/s) - ‘pipeline-documentation.md’ saved [4169/4169]



###Check to make sure looks good

In [4]:
import markdown
from IPython.display import HTML
pipe_html = markdown.markdown(pipe_md_content)
display(HTML(pipe_html))

In [5]:
pipe_md_content

'# Titanic Data Pipeline Documentation\n\n## Pipeline Overview\nThis pipeline preprocesses the Titanic dataset to prepare it for machine learning modeling. It handles categorical encoding, target encoding, outlier detection and treatment, feature scaling, and missing value imputation.\n\n<img src="https://github.com/fickas/asynch_models/blob/main/transform_pipeline.png?raw=true" width="70%" alt="Pipeline Diagram">\n\n## Step-by-Step Design Choices\n\n### 1. Gender Mapping (`map_gender`)\n- **Transformer:** `CustomMappingTransformer(\'Gender\', {\'Male\': 0, \'Female\': 1})`\n- **Design Choice:** Binary encoding of gender with female as 1 and male as 0\n- **Rationale:** Simple categorical mapping that preserves the binary nature of the feature without increasing dimensionality\n\n### 2. Class Mapping (`map_class`)\n- **Transformer:** `CustomMappingTransformer(\'Class\', {\'Crew\': 0, \'C3\': 1, \'C2\': 2, \'C1\': 3})`\n- **Design Choice:** Ordinal encoding of passenger class from lowest

##Bring in models

Comes from Tuning notebook. You will need to load the 4 models and 4 threshold tables.

In [6]:
#you will change this and all urls in your final project

model_path = 'fickas/asynch_models/main/'

In [7]:
#load trained lgb model
full_path = f'https://raw.githubusercontent.com/{model_path}final_lgb_model.joblib'
!rm 'final_lgb_model.joblib'
!wget $full_path
lgb_model = load('final_lgb_model.joblib')


rm: cannot remove 'final_lgb_model.joblib': No such file or directory
--2025-06-04 05:41:12--  https://raw.githubusercontent.com/fickas/asynch_models/main/final_lgb_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13620 (13K) [application/octet-stream]
Saving to: ‘final_lgb_model.joblib’

final_lgb_model.job 100%[===================>]  13.30K  --.-KB/s    in 0.001s  

2025-06-04 05:41:12 (22.1 MB/s) - ‘final_lgb_model.joblib’ saved [13620/13620]



In [8]:
#load trained logistic regression model
full_path = f'https://raw.githubusercontent.com/{model_path}final_logreg_model.joblib'
!rm 'final_logreg_model.joblib'
!wget $full_path
logreg_model = load('final_logreg_model.joblib')


rm: cannot remove 'final_logreg_model.joblib': No such file or directory
--2025-06-04 05:41:18--  https://raw.githubusercontent.com/fickas/asynch_models/main/final_logreg_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3546 (3.5K) [application/octet-stream]
Saving to: ‘final_logreg_model.joblib’

final_logreg_model. 100%[===================>]   3.46K  --.-KB/s    in 0s      

2025-06-04 05:41:19 (44.1 MB/s) - ‘final_logreg_model.joblib’ saved [3546/3546]



In [9]:
##load trained knn model
full_path = f'https://raw.githubusercontent.com/{model_path}final_knn_model.joblib'
!rm 'final_knn_model.joblib'
!wget $full_path
knn_model = load('final_knn_model.joblib')


rm: cannot remove 'final_knn_model.joblib': No such file or directory
--2025-06-04 05:41:19--  https://raw.githubusercontent.com/fickas/asynch_models/main/final_knn_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124086 (121K) [application/octet-stream]
Saving to: ‘final_knn_model.joblib’

final_knn_model.job 100%[===================>] 121.18K  --.-KB/s    in 0.03s   

2025-06-04 05:41:19 (4.44 MB/s) - ‘final_knn_model.joblib’ saved [124086/124086]



In [10]:
##load trained ann model
full_path = f'https://raw.githubusercontent.com/{model_path}final_ann_model.keras'
!rm 'final_ann_model.keras'
!wget $full_path
ann_model = tf.keras.models.load_model('final_ann_model.keras', custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU()})

rm: cannot remove 'final_ann_model.keras': No such file or directory
--2025-06-04 05:41:19--  https://raw.githubusercontent.com/fickas/asynch_models/main/final_ann_model.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34918 (34K) [application/octet-stream]
Saving to: ‘final_ann_model.keras’

final_ann_model.ker 100%[===================>]  34.10K  --.-KB/s    in 0.01s   

2025-06-04 05:41:19 (3.15 MB/s) - ‘final_ann_model.keras’ saved [34918/34918]



In [11]:
ann_model.input_shape

(None, 6)

##Bring in threshold tables

Comes from Tuning notebook.

In [12]:
#now threshold tables - comes from tuning notebook. Change in final project.

logreg_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_logreg_thresholds.csv').round(2)
knn_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_knn_thresholds.csv').round(2)
lgb_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_lgb_thresholds.csv').round(2)
ann_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_ann_thresholds.csv').round(2)

##Build threshold tables as HTML

Once we have HTML version, can insert that in page.



In [13]:
lgb_table = lgb_thresholds.to_html(index=False, justify='center').replace('<td>','<td style="text-align: center;">')
logreg_table = logreg_thresholds.to_html(index=False, justify='center').replace('<td>','<td style="text-align: center;">')
knn_table = knn_thresholds.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')
ann_table = ann_thresholds.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')

##Now fitted transformer

Comes from Wrangling notebook.

In [14]:
!pip install dill
import dill as pickle

In [15]:
full_path = f'https://raw.githubusercontent.com/{model_path}final_fully_fitted_pipeline.pkl'
!rm 'final_fully_fitted_pipeline.pkl'
!wget $full_path
# fitted_transformer = load('final_fully_fitted_pipeline.pkl')
# depends on fickas library but is unused

rm: cannot remove 'final_fully_fitted_pipeline.pkl': No such file or directory
--2025-06-04 05:41:24--  https://raw.githubusercontent.com/fickas/asynch_models/main/final_fully_fitted_pipeline.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73086 (71K) [application/octet-stream]
Saving to: ‘final_fully_fitted_pipeline.pkl’

final_fully_fitted_ 100%[===================>]  71.37K  --.-KB/s    in 0.02s   

2025-06-04 05:41:24 (3.09 MB/s) - ‘final_fully_fitted_pipeline.pkl’ saved [73086/73086]



#Now Lime

Comes from Tuning notebook.

In [16]:
feature_names  = ['Age', 'Gender', 'Class', 'Married', 'Fare', 'Joined']  #change to match your own table in final project

In [17]:
%%capture
!pip install lime

In [18]:
import lime
from lime import lime_tabular


In [19]:
#get trained lime explainer
full_path = f'https://raw.githubusercontent.com/{model_path}lime_explainer.pkl'
!rm 'lime_explainer.pkl'
!wget $full_path
with open('lime_explainer.pkl', 'rb') as file:
    lime_explainer = pickle.load(file)

rm: cannot remove 'lime_explainer.pkl': No such file or directory
--2025-06-04 05:41:34--  https://raw.githubusercontent.com/fickas/asynch_models/main/lime_explainer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7845 (7.7K) [application/octet-stream]
Saving to: ‘lime_explainer.pkl’

lime_explainer.pkl  100%[===================>]   7.66K  --.-KB/s    in 0.001s  

2025-06-04 05:41:34 (12.1 MB/s) - ‘lime_explainer.pkl’ saved [7845/7845]



##Done loading from GitHub

At this point, you have all your hard-earned work loaded in the server and ready to use. At least you will when you get to the final project.

The final step is tailoring the web page and functions that deal with it to fit your specific dataset.

#Big pedagogical choice!

My goal is to give you a template for a professional looking page that you are happy to demo for a potential employer. I saw two choices I could make:

1. Bring in a prebuilt framework, e.g., Shadcn/UI, that has styling and scripts predefined for you.

2. Build styling and scripts by scratch.

Let me tell you why I went with the 2nd. I wanted to localize the changes you needed to make to my code to fit your new dataset. By building from scratch, I was able to do that. I was not able to do it by using pre-built frameworks.

##The Bad and Good news

The bad news is there is a lot of html/javascript code I used when starting from scratch. The good news is that you should not need to touch it. Further good news is that you might actually learn from it if you have time to study it a bit. I tried to comment it.

#Start with template

We will be building a single (complicated) page website from pure html/js. This is not normally the way you would build a modern website. You would typically load libraries (AKA Shadcn/UI) that contained much of the foundation for you. Instead we will build from scratch.

Second, we will reload the page on every response from user. Again, this is not something you would typically do on production web site, instead just updating pieces that need to change.

So why did I choose to do it my way? To make it easy for you to demo to someone, e.g., potential employer. I have set things up so you have to change only one function, and a few small pieces of html. And by making the page reload on each interaction, a lot easier to debug - you can make changes to the page without restarting the server.

#Break into pieces to make it a little more undserstandable

I'll show you the basic template page below. You will notice placeholders with form `%...%.` I will use Python functions to compute what gets inserted into these places. So this single template will grow quite large as we fill in pieces. Again, you do not have to worry about that, unless you want to.

Here is basic template.

In [20]:
# Main page template using a two-column layout
fpage_template = '''
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Pima Diabetes Prediction</title>
%enhanced_styling%
</head>

<body>
  <div class="container">
    <div class="left-column">
      <!-- Header -->
      <div class="page-header">
        <h1>Pima Diabetes Prediction</h1>
        <img src='https://images.pond5.com/female-doctor-stethoscope-listening-patients-footage-088587876_prevstill.jpeg' height=200>
      </div>

      <!-- Prediction Form -->
      <div class="form-panel">
        <h2>Enter Patient Information</h2>
        %form_section%
      </div>

      <!-- Results Section -->
      <div class="results-panel">
        %results_section%
      </div>
    </div>

    <div class="right-column" id="tableContainer">
      <!-- This space is intentionally left empty for threshold tables -->
      %threshold_tables%
    </div>
  </div>

  <script>
    /**
     * Improved toggle function with scrolling
     *
     * This function:
     * 1. Hides all tables
     * 2. If the clicked table was hidden, shows it and scrolls to it
     * 3. If the clicked table was already visible, keeps it hidden
     *
     * @param {string} tableId - The ID of the table to toggle
     */
  function toggleTable(tableId) {
    // Get the table element
    var table = document.getElementById(tableId);

    // Get all tables
    var tables = document.querySelectorAll('.table-wrapper');

    // Check if this table is currently visible (using computed style)
    var computedStyle = window.getComputedStyle(table);
    var isVisible = computedStyle.display !== 'none';

    // First, hide all tables
    tables.forEach(function(t) {
      t.style.display = 'none';
    });

    // If the table was not visible, show it and scroll to it
    if (!isVisible) {
      table.style.display = 'block';

      // Scroll the right column into view
      document.getElementById('tableContainer').scrollIntoView({
        behavior: 'smooth',
        block: 'start'
      });
    }
  // If it was visible, it will remain hidden (toggled off)
}
  </script>
</body>
</html>
'''

#Note on challenges

I have broken out changes into two challenges. However, it might be easier to lump them together. So do both changes and then show page to make sure it looks ok.

For grading, I am looking for a page that has both changes. If I see that, you are good to go.

#Challenge 1

You will be working with the Pima dataset (midterm 1). Change the titles and find an image that goes with this dataset and change the template above.

I see changes on lines 7, 16, and 17.

**Done**

## Set-up styling

This is a complicated function that defines styling to give the page a more professional look. I have left comments. I do not expect you to change it although you can if you want.

In [21]:
import markdown

def get_enhanced_styling():
    """
    Returns enhanced CSS styling for the demo page.

    This function contains all the CSS styling rules for the page, organized by section.
    Each section has detailed comments explaining what the styles control and how they work.

    Returns:
    --------
    str
        A string containing all CSS styling rules wrapped in <style> tags
    """
    return '''
    <style>
      /*********************************************
       * BASE STYLES
       * These styles apply to the entire document
       *********************************************/

      /*
       * Basic body styling
       * - Sets background color to light blue-gray
       * - Sets default text color to dark gray
       * - Uses modern, sans-serif font stack
       * - Sets comfortable line spacing
       * - Removes default margins and padding
       */
      body {
        background-color: #f0f4f8;  /* Light blue-gray background */
        color: #333;                /* Dark gray text */
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;  /* Modern font stack */
        line-height: 1.6;           /* Comfortable line spacing for readability */
        margin: 0;                  /* Remove default margin */
        padding: 0;                 /* Remove default padding */
      }

      /*********************************************
       * LAYOUT STRUCTURE
       * Controls the two-column layout of the page
       *********************************************/

      /*
       * Main container
       * - Creates a flex container for the two-column layout
       * - Ensures the container takes up at least the full viewport height
       */
      .container {
        display: flex;              /* Use flexbox for layout */
        min-height: 100vh;          /* Ensure container is at least full viewport height */
      }

      /*
       * Left column
       * - Contains the form and results
       * - Takes up 45% of the container width
       * - Has padding around all sides
       */
      .left-column {
        width: 45%;                 /* Set column width to 45% of container */
        padding: 20px;              /* Add spacing around content */
        box-sizing: border-box;     /* Include padding in width calculation */
      }

      /*
       * Right column
       * - Reserved for threshold tables
       * - Takes up 55% of the container width
       * - Has padding around all sides
       * - Position is relative for child positioning
       */
      .right-column {
        width: 55%;                 /* Set column width to 55% of container */
        padding: 20px;              /* Add spacing around content */
        box-sizing: border-box;     /* Include padding in width calculation */
        position: relative;         /* For positioning tables */
      }

      /*********************************************
       * HEADER SECTION
       * Styles for the page header and title
       *********************************************/

      /*
       * Page header container
       * - Has a blue gradient background
       * - White text
       * - Rounded corners
       * - Bottom margin to separate from content below
       * - Subtle shadow for depth
       */
      .page-header {
        background: linear-gradient(135deg, #23283a 0%, #2b5876 100%);  /* Gradient background */
        color: white;               /* White text */
        padding: 20px;              /* Internal spacing */
        border-radius: 8px;         /* Rounded corners */
        margin-bottom: 25px;        /* Space below header */
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
      }

      /*
       * Main page heading (h1)
       * - No margin to avoid extra space
       * - White text color
       * - Larger font size
       * - Medium font weight
       */
      .page-header h1 {
        margin: 0;                  /* Remove default margin */
        color: white;               /* White text */
        font-size: 32px;            /* Larger text */
        font-weight: 500;           /* Medium weight */
      }

      /*
       * Header image styling
       * - Responsive width
       * - Maintain aspect ratio
       * - Rounded corners
       * - Space above image
       */
      .page-header img {
        max-width: 100%;            /* Responsive width */
        height: auto;               /* Maintain aspect ratio */
        border-radius: 4px;         /* Rounded corners */
        margin-top: 15px;           /* Space above image */
      }

      /*********************************************
       * PANEL STYLING
       * Styles for the form and results panels
       *********************************************/

      /*
       * Form panel styling
       * - White background
       * - Rounded corners
       * - Internal padding
       * - Subtle shadow
       * - Bottom margin for spacing
       */
      .form-panel {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        padding: 25px;              /* Internal spacing */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        margin-bottom: 25px;        /* Space below panel */
      }

      /*
       * Form panel heading
       * - No top margin to avoid extra space
       * - Blue text color
       * - Bottom border for visual separation
       * - Padding below text for spacing from border
       * - Medium font weight
       */
      .form-panel h2 {
        margin-top: 0;              /* Remove top margin */
        color: #2b5876;             /* Blue heading text */
        border-bottom: 2px solid #f0f4f8;  /* Light bottom border */
        padding-bottom: 10px;       /* Space between text and border */
        font-weight: 500;           /* Medium weight */
      }

      /*
       * Results panel styling
       * - White background
       * - Rounded corners
       * - Internal padding
       * - Subtle shadow
       */
      .results-panel {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        padding: 25px;              /* Internal spacing */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
      }

      /*
       * Results panel heading
       * - No top margin to avoid extra space
       * - Blue text color
       * - Medium font weight
       */
      .results-panel h2 {
        margin-top: 0;              /* Remove top margin */
        color: #2b5876;             /* Blue heading text */
        font-weight: 500;           /* Medium weight */
      }

      /*********************************************
       * FORM CONTROL STYLING
       * Styles for form inputs, dropdowns, and buttons
       *********************************************/

      /*
       * Text input styling
       * - Full width of container
       * - Internal padding
       * - Light border
       * - Rounded corners
       * - Box sizing to include padding in width
       * - Comfortable font size
       * - Top margin for spacing from label
       */
      input[type="text"] {
        width: 100%;                /* Full width */
        padding: 10px 15px;         /* Vertical and horizontal padding */
        border: 1px solid #ddd;     /* Light gray border */
        border-radius: 4px;         /* Rounded corners */
        box-sizing: border-box;     /* Include padding in width */
        font-size: 16px;            /* Comfortable font size */
        margin-top: 6px;            /* Space from label above */
      }

      /*
       * Dropdown select styling
       * - Full width of container
       * - Internal padding
       * - Light border
       * - Rounded corners
       * - White background
       * - Box sizing to include padding in width
       * - Comfortable font size
       * - Top margin for spacing from label
       * - Remove default appearance for custom styling
       * - Add custom dropdown arrow
       */
      select {
        width: 100%;                /* Full width */
        padding: 10px 15px;         /* Vertical and horizontal padding */
        border: 1px solid #ddd;     /* Light gray border */
        border-radius: 4px;         /* Rounded corners */
        background-color: white;    /* White background */
        box-sizing: border-box;     /* Include padding in width */
        font-size: 16px;            /* Comfortable font size */
        margin-top: 6px;            /* Space from label above */
        appearance: none;           /* Remove default browser styling */

        /* Add custom dropdown arrow as background image */
        background-image: url("data:image/svg+xml;charset=US-ASCII,%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22292.4%22%20height%3D%22292.4%22%3E%3Cpath%20fill%3D%22%23007CB2%22%20d%3D%22M287%2069.4a17.6%2017.6%200%200%200-13-5.4H18.4c-5%200-9.3%201.8-12.9%205.4A17.6%2017.6%200%200%200%200%2082.2c0%205%201.8%209.3%205.4%2012.9l128%20127.9c3.6%203.6%207.8%205.4%2012.8%205.4s9.2-1.8%2012.8-5.4L287%2095c3.5-3.5%205.4-7.8%205.4-12.8%200-5-1.9-9.2-5.5-12.8z%22%2F%3E%3C%2Fsvg%3E");
        background-repeat: no-repeat;
        background-position: right 10px center;  /* Position arrow on right side */
        background-size: 12px;      /* Size of arrow */
      }

      /*
       * Form group container
       * - Adds bottom margin for spacing between form elements
       */
      .form-group {
        margin-bottom: 20px;        /* Space below group */
      }

      /*
       * Form label styling
       * - Display as block for layout
       * - Bottom margin for spacing
       * - Medium font weight
       * - Blue text color for consistency
       */
      label {
        display: block;             /* Make labels block elements */
        margin-bottom: 6px;         /* Space below label */
        font-weight: 500;           /* Medium weight */
        color: #2b5876;             /* Blue text */
      }

      /*
       * Submit button styling
       * - Blue/purple gradient background
       * - White text
       * - No border
       * - Comfortable padding
       * - Larger font size
       * - Rounded corners
       * - Pointer cursor on hover
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       */
      .submit-btn {
        background: linear-gradient(135deg, #2b5876 0%, #4e4376 100%);  /* Gradient background */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 12px 24px;         /* Vertical and horizontal padding */
        font-size: 18px;            /* Larger text */
        border-radius: 4px;         /* Rounded corners */
        cursor: pointer;            /* Hand cursor on hover */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Submit button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .submit-btn:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*********************************************
       * RESULTS BUTTONS STYLING
       * Styles for the model result buttons
       *********************************************/

      /*
       * Models list container
       * - Uses flexbox for layout
       * - Allows wrapping to multiple lines if needed
       * - Adds spacing between buttons
       * - Bottom margin for spacing
       */
      .models-list {
        display: flex;              /* Use flexbox for layout */
        flex-wrap: wrap;            /* Allow wrapping to multiple lines */
        gap: 10px;                  /* Spacing between buttons */
        margin-bottom: 20px;        /* Space below button group */
      }

      /*
       * Model button styling
       * - Blue/purple gradient background
       * - White text
       * - No border
       * - Comfortable padding
       * - Rounded corners
       * - Appropriate font size
       * - Pointer cursor on hover
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       * - Only take up needed space (not full width)
       * - Center text
       */
      .model-btn {
        background: linear-gradient(135deg, #2b5876 0%, #4e4376 100%);  /* Gradient background */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 10px 16px;         /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        font-size: 16px;            /* Comfortable font size */
        cursor: pointer;            /* Hand cursor on hover */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
        flex: 0 0 auto;             /* Only take up needed space */
        text-align: center;         /* Center text */
      }

      /*
       * Model button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .model-btn:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*
       * Documentation button styling
       * - Blue gradient background (different from model buttons)
       * - White text
       * - No border
       * - Appropriate padding
       * - Rounded corners
       * - Comfortable font size
       * - Pointer cursor on hover
       * - Left margin for spacing
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       */
      .doc-button {
        background: linear-gradient(135deg, #3a7bd5 0%, #00d2ff 100%);  /* Blue gradient */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 8px 15px;          /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        font-size: 16px;            /* Comfortable font size */
        cursor: pointer;            /* Hand cursor on hover */
        margin-left: 10px;          /* Space to the left */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Documentation button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .doc-button:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*********************************************
       * MODAL STYLING
       * Styles for the documentation popup modal
       *********************************************/

      /*
       * Modal container
       * - Hidden by default
       * - Fixed position covering the entire viewport
       * - High z-index to appear above other content
       * - Semi-transparent black background overlay
       * - Scrollable if content is too tall
       */
      .modal {
        display: none;              /* Hidden by default */
        position: fixed;            /* Fixed position */
        z-index: 1000;              /* High z-index to be on top */
        left: 0;                    /* Align to left edge */
        top: 0;                     /* Align to top edge */
        width: 100%;                /* Full width */
        height: 100%;               /* Full height */
        background-color: rgba(0,0,0,0.5);  /* Semi-transparent background */
        overflow: auto;             /* Allow scrolling if needed */
      }

      /*
       * Modal content container
       * - White background
       * - Centered with margin
       * - Internal padding
       * - Rounded corners
       * - Limited width and height
       * - Scrollable if content is too tall
       * - Relative position for close button
       * - Shadow for depth
       */
      .modal-content {
        background-color: #fefefe;  /* White background */
        margin: 2% auto;            /* Center horizontally with space at top */
        padding: 20px;              /* Internal spacing */
        border-radius: 8px;         /* Rounded corners */
        width: 80%;                 /* Width relative to viewport */
        max-width: 900px;           /* Maximum width */
        max-height: 90vh;           /* Maximum height relative to viewport */
        overflow-y: auto;           /* Vertical scrolling if needed */
        position: relative;         /* For positioning close button */
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);  /* Shadow for depth */
      }

      /*
       * Close button styling
       * - Absolute position in top right corner
       * - White text on blue background
       * - Bold text
       * - Pointer cursor on hover
       * - Comfortable padding
       * - Rounded corners
       * - Shadow for depth
       * - Smooth transition for hover effects
       */
      .close-button {
        position: absolute;         /* Absolute position */
        top: 15px;                  /* Top spacing */
        right: 20px;                /* Right spacing */
        color: #fff;                /* White text */
        background-color: #2b5876;  /* Blue background */
        font-size: 16px;            /* Comfortable font size */
        font-weight: bold;          /* Bold text */
        cursor: pointer;            /* Hand cursor on hover */
        padding: 5px 15px;          /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        box-shadow: 0 2px 4px rgba(0,0,0,0.2);  /* Shadow for depth */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Close button hover state
       * - Darker background color
       * - Larger shadow for "lifting" effect
       */
      .close-button:hover {
        background-color: #23283a;  /* Darker blue on hover */
        box-shadow: 0 4px 8px rgba(0,0,0,0.3);  /* Larger shadow */
      }

      /*********************************************
       * TABLE STYLING
       * Styles for the threshold tables
       *********************************************/

      /*
       * Table wrapper container
       * - Hidden by default
       * - Sticky position to stay visible when scrolling
       * - Top spacing when sticky
       */
      .table-wrapper {
        display: none;              /* Hidden by default */
        position: sticky;           /* Sticky positioning */
        top: 20px;                  /* Top spacing when sticky */
      }

      /*
       * Common styling for both table columns
       * - White background
       * - Rounded corners
       * - Subtle shadow
       * - Internal padding
       * - Bottom margin for spacing
       */
      .table1, .table2 {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        padding: 15px;              /* Internal spacing */
        margin-bottom: 20px;        /* Space below */
      }

      /*
       * Left table column
       * - Float left
       * - 45% width of container
       */
      .table1 {
        float: left;                /* Float left */
        width: 45%;                 /* 45% width */
      }

      /*
       * Right table column
       * - Float right
       * - 45% width of container
       * - Left margin for spacing from left column
       */
      .table2 {
        float: right;               /* Float right */
        width: 45%;                 /* 45% width */
        margin-left: 10px;          /* Space from left column */
      }

      /*
       * Ensemble result styling
       * - Larger font size
       * - Medium font weight
       * - Blue text color
       * - Top margin for spacing
       * - Top padding for spacing
       * - Top border for visual separation
       * - Clear both floats to appear below buttons
       */
      .ensemble-result {
        font-size: 24px;            /* Larger text */
        font-weight: 500;           /* Medium weight */
        color: #2b5876;             /* Blue text */
        margin-top: 15px;           /* Space above */
        padding: 10px;              /* Padding */
        border-top: 2px solid #f0f4f8;  /* Light top border */
        clear: both;                /* Clear floats */
      }

      /*********************************************
       * MARKDOWN CONTENT STYLING
       * Styles for the rendered markdown in the documentation modal
       *********************************************/

      /*
       * Markdown container styling
       * - Modern font stack
       * - Comfortable line height
       */
      #markdown-content {
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Arial, sans-serif;
        line-height: 1.6;           /* Comfortable line height */
      }

      /*
       * Markdown headings
       * - Blue text color
       */
      #markdown-content h1, #markdown-content h2 {
        color: #2c3e50;             /* Blue text */
      }

      /*
       * Markdown h2 headings
       * - Bottom border for visual separation
       * - Bottom padding for spacing from border
       */
      #markdown-content h2 {
        border-bottom: 1px solid #eee;  /* Light bottom border */
        padding-bottom: 5px;        /* Space between text and border */
      }

      /*
       * Markdown code blocks
       * - Light gray background
       * - Padding for spacing
       * - Rounded corners
       * - Monospace font
       */
      #markdown-content code {
        background-color: #f5f5f5;  /* Light gray background */
        padding: 2px 4px;           /* Vertical and horizontal padding */
        border-radius: 3px;         /* Rounded corners */
        font-family: monospace;     /* Monospace font */
      }

      /*
       * Markdown lists
       * - Left padding for indentation
       */
      #markdown-content ul, #markdown-content ol {
        padding-left: 25px;         /* Left padding */
      }

      /*
       * Markdown paragraphs
       * - Bottom margin for spacing
       */
      #markdown-content p {
        margin-bottom: 16px;        /* Space below */
      }

      /*
       * Markdown blockquotes
       * - Left and right padding
       * - Gray text color
       * - Left border for visual distinction
       * - Bottom margin for spacing
       */
      #markdown-content blockquote {
        padding: 0 1em;             /* Horizontal padding */
        color: #6a737d;             /* Gray text */
        border-left: 0.25em solid #dfe2e5;  /* Left border */
        margin: 0 0 16px 0;         /* Bottom margin */
      }

      /*
       * Markdown tables
       * - Collapse borders
       * - Full width
       * - Bottom margin for spacing
       */
      #markdown-content table {
        border-collapse: collapse;  /* Collapse borders */
        width: 100%;                /* Full width */
        margin-bottom: 16px;        /* Space below */
      }

      /*
       * Markdown table cells
       * - Padding for spacing
       * - Light border
       */
      #markdown-content table th, #markdown-content table td {
        padding: 6px 13px;          /* Vertical and horizontal padding */
        border: 1px solid #dfe2e5;  /* Light border */
      }

      /*
       * Markdown table alternating rows
       * - Light gray background for even rows
       */
      #markdown-content table tr:nth-child(2n) {
        background-color: #f6f8fa;  /* Light gray background */
      }

      /*********************************************
       * RESPONSIVE DESIGN
       * Adjustments for smaller screens
       *********************************************/

      /*
       * Media query for screens smaller than 1200px
       * - Changes layout from two columns to one column
       * - Makes both columns full width
       * - Changes right column positioning
       */
      @media (max-width: 1200px) {
        .container {
          flex-direction: column;   /* Stack columns vertically */
        }

        .left-column, .right-column {
          width: 100%;              /* Full width */
        }

        .right-column {
          position: static;         /* Normal positioning */
        }
      }
    </style>
    '''


##These are helper functions

Again, you should not have to change them although you are welcome to do so.

In [22]:

def text_input(name, label, placeholder=""):
    """
    Generate HTML for a styled text input field with label.

    This function creates a form group containing a label and text input field.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    name : str
        The input field name (used for form submission and label connection)
    label : str
        The descriptive text for the label
    placeholder : str, optional
        Placeholder text shown in the input when empty

    Returns:
    --------
    str
        HTML markup for the label and input field wrapped in a form group
    """
    return f'''
    <div class="form-group">
      <label for="{name}">{label}</label>
      <input type="text" id="{name}" name="{name}_field" placeholder="{placeholder}">
    </div>
    '''

def dropdown_select(name, label, options):
    """
    Generate HTML for a styled dropdown select with label.

    This function creates a form group containing a label and select dropdown.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    name : str
        The select field name (used for form submission and label connection)
    label : str
        The descriptive text for the label
    options : dict
        Dictionary of {value: display_text} pairs for the dropdown options

    Returns:
    --------
    str
        HTML markup for the label and select dropdown wrapped in a form group
    """
    options_html = ""
    for value, text in options.items():
        options_html += f'<option value="{value}">{text}</option>\n'

    return f'''
    <div class="form-group">
      <label for="{name}">{label}</label>
      <select id="{name}" name="{name}">
        {options_html}
      </select>
    </div>
    '''

def submit_button(label="Submit"):
    """
    Generate HTML for a styled submit button.

    This function creates a submit button for a form.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    label : str, optional
        The text displayed on the button (default: "Submit")

    Returns:
    --------
    str
        HTML markup for the submit button
    """
    return f'''
    <button type="submit" class="submit-btn">{label}</button>
    '''


In [23]:
def get_results_section():
    """
    Generate HTML for the results section with buttons.

    This function creates the results section of the page, including:
    - A heading
    - The pipeline documentation button (inserted via placeholder)
    - A row data display area (inserted via placeholder)
    - Buttons for each model's results
    - An ensemble result display

    The styling is handled by CSS classes defined in get_enhanced_styling().

    Returns:
    --------
    str
        HTML markup for the complete results section with placeholders
    """
    return '''
    <h2>
      Results for data:
    </h2>
    %pipeline_docs%
    <h3>%row_data%</h3>

    <div class="models-list">
      <button class="model-btn" onclick="toggleTable('lgb')">
        LGB: %lgb%
      </button>

      <button class="model-btn" onclick="toggleTable('knn')">
        KNN: %knn%
      </button>

      <button class="model-btn" onclick="toggleTable('logreg')">
        LogReg: %logreg%
      </button>

      <button class="model-btn" onclick="toggleTable('ann')">
        ANN: %ann%
      </button>
    </div>

    <div class="ensemble-result">
      Ensemble: %ensemble%
    </div>
    '''

def get_threshold_tables_section():
    """
    Generate HTML for the threshold tables section.

    This function creates the table containers for all model results.
    Each model has two tables: a threshold table and a LIME explanation table.
    By default, all tables are hidden and will only be shown when their
    corresponding button is clicked.

    The styling is handled by CSS classes defined in get_enhanced_styling().

    Returns:
    --------
    str
        HTML markup for all the model result tables with placeholders
    """
    return '''
    <!-- LGB Model Tables -->
    <div id="lgb" class='table-wrapper'>
      <div class="table1">
        <center><h2>LGB Threshold Table</h2></center>
        %lgb_table%
      </div>
      <div class="table2">
      <center><h2>LGB Lime Explanation</h2></center>
        %lgb_lime_table%
      </div>
    </div>

    <!-- KNN Model Tables -->
    <div id="knn" class='table-wrapper'>
      <div class="table1">
      <center><h2>KNN Threshold Table</h2></center>
        %knn_table%
      </div>
      <div class="table2">
      <center><h2>KNN Lime Explanation</h2></center>
        %knn_lime_table%
      </div>
    </div>

    <!-- LogReg Model Tables -->
    <div id="logreg" class='table-wrapper'>
      <div class="table1">
      <center><h2>Logistic Regression Threshold Table</h2></center>
        %logreg_table%
      </div>
      <div class="table2">
      <center><h2>Logistic Regression Lime Explanation</h2></center>
        %logreg_lime_table%
      </div>
    </div>

    <!-- ANN Model Tables -->
    <div id="ann" class='table-wrapper'>
      <div class="table1">
      <center><h2>ANN Threshold Table</h2></center>
        %ann_table%
      </div>
      <div class="table2">
      <center><h2>ANN Lime Explanation</h2></center>
        %ann_lime_table%
      </div>
    </div>
    '''

def get_pipeline_documentation(md_content):

    # Convert markdown to HTML using Python-Markdown
    html_content = markdown.markdown(
        md_content,
        extensions=['tables', 'fenced_code', 'nl2br']
    )

    # Create complete HTML for popup with styled content and JS functionality
    documentation_html = f"""
    <!-- Button to open modal -->
    <button class="doc-button" id="openDocBtn">See Preprocessing Steps</button>

    <!-- The Modal/Popup -->
    <div id="docModal" class="modal">
        <!-- Modal content -->
        <div class="modal-content">
            <span class="close-button" id="closeModal">CLOSE</span>
            <div id="markdown-content">
                {html_content}
            </div>
        </div>
    </div>

    <!-- JavaScript for modal functionality -->
    <script>
        // Get modal elements
        const modal = document.getElementById("docModal");
        const btn = document.getElementById("openDocBtn");
        const closeBtn = document.getElementById("closeModal");

        // Open modal when button is clicked
        btn.onclick = function() {{
            modal.style.display = "block";
        }}

        // Close modal ONLY when CLOSE is clicked
        closeBtn.onclick = function() {{
            modal.style.display = "none";
        }}

        // Prevent modal from closing when clicking on the content
        document.querySelector(".modal-content").onclick = function(event) {{
            event.stopPropagation();
        }}
    </script>
    """

    return documentation_html

pipeline_docs_html = get_pipeline_documentation(pipe_md_content)


#This actually starts gluing things together

You can see that there are a series of string `replace` methods. They start with the `fpage_template` then end up in `fpage` as full website.

In [24]:
def create_template_page(config, fpage_template, pipeline_docs_html, lgb_table, logreg_table, knn_table, ann_table):
    """
    Main function to generate the complete Titanic prediction page.

    This function:
    1. Gets all the components (styling, form, results, tables, docs)
    2. Replaces all placeholders in the template
    3. Returns the complete HTML page

    Parameters:
    -----------

    Returns:
    --------
    str
        The complete HTML for the Titanic prediction page
    """

    # Compute all the pieces
    form_html = complete_form(config)
    results_section_html = get_results_section()
    threshold_tables_html = get_threshold_tables_section()
    enhanced_styling = get_enhanced_styling()

    # Replace all placeholders
    fpage = fpage_template.replace('%enhanced_styling%', enhanced_styling)
    fpage = fpage.replace('%form_section%', form_html)
    fpage = fpage.replace('%results_section%', results_section_html)
    fpage = fpage.replace('%threshold_tables%', threshold_tables_html)
    fpage = fpage.replace('%pipeline_docs%', pipeline_docs_html)

    # These threshold tables do not change so add them now
    fpage = fpage.replace('%lgb_table%', lgb_table)
    fpage = fpage.replace('%logreg_table%', logreg_table)
    fpage = fpage.replace('%knn_table%', knn_table)
    fpage = fpage.replace('%ann_table%', ann_table)

    return fpage

#This you have to change

This is set-up for the Titanic. You will need to make changes here for one of the challenges.

In essence, this is the result of my attempts to localize changes for you.

In [35]:
def get_dataset_config():
    """
    Centralized configuration for Titanic dataset fields.
    This single function defines both form elements and data processing.

    Returns a dictionary where each key is a field name, and each value is
    a dictionary of properties for that field.
    """
    return {
        "Glucose": {
            "form_field": "glucose_field",
            "label": "Enter Glucose",
            "placeholder": "Unknown",
            "type": "numeric",
            "input_type": "text",
            "column_name": "Glucose",
            "process": lambda x: int(x) if x.isdigit() and int(x) > 0 else np.nan,
        },
        "BloodPressure": {
            "form_field": "blood_pressure_field",
            "label": "Enter Blood Pressure",
            "placeholder": "Unknown",
            "type": "categorical",
            "input_type": "select",
            "column_name": "BloodPressure",
            "process": lambda x: int(x) if x != "unknown" else np.nan,
            "options": {
                "Unknown": "unknown",
                "Low": "low",
                "Medium": "medium",
                "High": "high"
            }
        },
        "SkinThickness": {
            "form_field": "skin_thickness_field",
            "label": "Enter Skin Thickness",
            "placeholder": "Unknown",
            "type": "numeric",
            "input_type": "text",
            "column_name": "SkinThickness",
            "process": lambda x: int(x) if x.isdigit() and int(x) > 0 else np.nan,
        },
        "Region": {
            "form_field": "region_field",
            "label": "Enter Region",
            "placeholder": "Unknown",
            "type": "categorical",
            "input_type": "select",
            "column_name": "Region",
            "process": lambda x: x if x != "unknown" else np.nan,
            "options": {
                "Unknown": "unknown",
                "A": "A",
                "B": "B",
                "C": "C",
                "D": "D"
            }
        }
    }

# Get dataset configuration
config = get_dataset_config()


#This function is key

It takes the config file you build and integrates it with the last piece you need, the set of questions to ask the user.

In [37]:
def complete_form(config, form_id="row_info", action="data", method="POST"):
    """
    Generate form HTML using the dataset configuration.
    """
    form_elements = []

    # Create HTML for each field based on its type
    for field_id, field_config in config.items():
        if field_config["input_type"] == "text":
            form_elements.append(
                text_input(
                    field_config["form_field"],
                    field_config["label"],
                    field_config.get("placeholder", "")
                )
            )
        elif field_config["input_type"] == "select":
            form_elements.append(
                dropdown_select(
                    field_config["form_field"],
                    field_config["label"],
                    field_config["options"]
                )
            )

    # Build the complete form
    form_html = f'''
    <form id="{form_id}" action="{action}" method="{method}">
        <input type='hidden' id='hidden1' value='hidden value'/>
        {''.join(form_elements)}
        {submit_button("Evaluate")}
    </form>
    '''

    return form_html

fpage = create_template_page(config, fpage_template, pipeline_docs_html, lgb_table, logreg_table, knn_table, ann_table)  #start the glueing process

#Simulate Lime tables

We cannot generate Lime tables here - need actual runtime server.

In [38]:
#the Lime tables (filled in at runtime)
fpage = fpage.replace('%lgb_lime_table%', '<center><h2>Lime table will go here</h2></center>')
fpage = fpage.replace('%knn_lime_table%', '<center><h2>Lime table will go here</h2></center>')
fpage = fpage.replace('%logreg_lime_table%', '<center><h2>Lime table will go here</h2></center>')
fpage = fpage.replace('%ann_lime_table%', '<center><h2>Lime table will go here</h2></center>')

#Simulate prediction results

We cannot generate actual predictions - need actual runtime server.

In [39]:
#the predictions (simulated)
fpage = fpage.replace('%ann%', '.81')
fpage = fpage.replace('%logreg%', '.80')
fpage = fpage.replace('%knn%', '.79')
fpage = fpage.replace('%lgb%', '.82')
fpage = fpage.replace('%ensemble%', f'{(.81+.80+.79+.82)/4}')


##Below is not part of server

Just for showing how page looks here.

In [40]:
!rm fpage.html
f = open("fpage.html", "w")
f.write(fpage)
f.close()


###Don't click Evaluate

It assumes you have runtime server.

In [41]:
import IPython
IPython.display.HTML(filename='//content/fpage.html')

threshold,precision,recall,f1,auc,accuracy
0.00,0.43,1.00,0.60,0.50,0.43
0.06,0.43,1.00,0.60,0.50,0.43
0.11,0.44,1.00,0.61,0.50,0.44
0.17,0.45,1.00,0.62,0.53,0.46
0.22,0.50,0.96,0.66,0.61,0.56
0.28,0.56,0.90,0.69,0.68,0.65
0.33,0.63,0.83,0.72,0.73,0.72
0.39,0.63,0.82,0.71,0.73,0.71
0.44,0.71,0.63,0.67,0.72,0.73
0.50,0.81,0.61,0.69,0.75,0.77


In [42]:
foobar()  #to stop run all here

NameError: name 'foobar' is not defined

#Challenge 2

Create a new Diabetes predictor frontend page using Midterm 1 as a guide. Here is the full table from Midterm 1.

In [43]:
url = 'https://www.dropbox.com/s/lq431y13g3zc6lr/w22_pima_midterm1.csv?raw=1'
pima_df = pd.read_csv(url)
pima_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Region,Outcome
0,6,148.0,medium,35.0,NaN,33.6,0.627,50,C,Yes
1,1,85.0,medium,29.0,NaN,26.6,0.351,31,B,No
2,8,183.0,medium,NaN,NaN,23.3,NaN,32,D,Yes
3,1,89.0,medium,23.0,94.0,28.1,0.167,21,B,No
4,0,137.0,low,35.0,168.0,43.1,2.288,33,B,Yes
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,medium,48.0,180.0,32.9,0.171,63,D,No
764,2,122.0,medium,27.0,NaN,36.8,0.340,27,A,No
765,5,121.0,medium,23.0,112.0,26.2,0.245,30,B,No
766,1,126.0,low,NaN,NaN,30.1,0.349,47,A,Yes


###Assume we trim the columns down to 4

* Glucose
* BloodPressure
*	SkinThickness
* Region

You can ignore the rest. So user only has to supply values for those 4. Remember the user will want to input values pre-transformed, i.e., as they appear in table above.

##Step 1. Concentrate on `get_dataset_config` function

It is really the only place I am asking you to change for now.

And go ahead and change it above. Keep rendering it until you are happy.

Hint: I would copy and paste the two input types, select and text, and then make slight edit changes to fit Diabetes problem.

##Step 2. Make sure it looks ok

Should now see results from Challenge 1 and Challenge 2 showing.

Note you will have to rerun challenge 1 code to build up fpage again, but now with new input fields. Then can run code below to check if looks ok.

In [44]:
!rm fpage.html
f = open("fpage.html", "w")
f.write(fpage)
f.close()


###Don't click Evaluate

It assumes you have runtime server.

In [45]:
IPython.display.HTML(filename='//content/fpage.html')

threshold,precision,recall,f1,auc,accuracy
0.00,0.43,1.00,0.60,0.50,0.43
0.06,0.43,1.00,0.60,0.50,0.43
0.11,0.44,1.00,0.61,0.50,0.44
0.17,0.45,1.00,0.62,0.53,0.46
0.22,0.50,0.96,0.66,0.61,0.56
0.28,0.56,0.90,0.69,0.68,0.65
0.33,0.63,0.83,0.72,0.73,0.72
0.39,0.63,0.82,0.71,0.73,0.71
0.44,0.71,0.63,0.67,0.72,0.73
0.50,0.81,0.61,0.69,0.75,0.77


#In the next videos I will go over the actual server

That said, most of the action for the server is in editing the two steps above to get it to fit your dataset.

This is the last chapter assignment. What is due next is your final project.